In [3]:
# ======================================
# Cell 1: Import Libraries
# ======================================
import pandas as pd
import numpy as np


In [13]:
# ======================================
# Cell 2: Create Ratings Dataset
# ======================================

# Each row represents a user rating for an item.
ratings_dict = {
    "userId": [1, 1, 2, 2, 3, 3, 4, 4],  # Users
    "itemId": ["A", "B", "A", "C", "B", "D", "A", "D"],  # Items
    "rating": [5, 4, 4, 5, 2, 3, 3, 4],  # Ratings (1-5)
}

# Create DataFrame
df = pd.DataFrame(ratings_dict)
print("Ratings Data:")
df


Ratings Data:


,userId,itemId,rating
0,1,A,5
1,1,B,4
2,2,A,4
3,2,C,5
4,3,B,2
5,3,D,3
6,4,A,3
7,4,D,4


In [12]:
# ======================================
# Cell 3: Create User-Item Matrix
# ======================================

# Convert to user-item matrix with 0 for missing ratings
user_item_matrix = df.pivot_table(
    index='userId',
    columns='itemId',
    values='rating'
).fillna(0)

print("User-Item Matrix:")
user_item_matrix


User-Item Matrix:


itemId,A,B,C,D
userId,,,,
1,5.0,4.0,0.0,0.0
2,4.0,0.0,5.0,0.0
3,0.0,2.0,0.0,3.0
4,3.0,0.0,0.0,4.0


In [6]:
# ======================================
# Cell 4: Apply Singular Value Decomposition (SVD)
# ======================================

# Decompose the user-item matrix
U, sigma, Vt = np.linalg.svd(user_item_matrix, full_matrices=False)

# Convert sigma (1D) into diagonal matrix
sigma = np.diag(sigma)


In [10]:
# ======================================
# Cell 5: Reconstruct Predicted Ratings
# ======================================

# Predicted ratings = U * Sigma * Vt
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

# Convert to DataFrame
pred_df = pd.DataFrame(
    predicted_ratings,
    index=user_item_matrix.index,
    columns=user_item_matrix.columns
)

print("Predicted Ratings Matrix:")
pred_df


Predicted Ratings Matrix:


itemId,A,B,C,D
userId,,,,
1,5.0,4.000000e+00,-5.551115e-16,-9.992007e-16
2,4.0,-1.332268e-15,5.000000e+00,8.881784e-16
3,0.0,2.000000e+00,7.771561e-16,3.000000e+00
4,3.0,-1.110223e-16,-4.440892e-16,4.000000e+00


In [8]:
# ======================================
# Cell 6: Recommendation Function
# ======================================

def recommend_items(user_id, n=2):
    """
    Recommend top 'n' items for a given user based on predicted ratings.
    """
    # Get predicted ratings for the user
    user_ratings = pred_df.loc[user_id]
    
    # Exclude items the user already rated
    already_rated = df[df['userId'] == user_id]['itemId'].values
    
    # Get top N highest predicted ratings
    recommendations = user_ratings.drop(already_rated).sort_values(ascending=False)[:n]
    
    return recommendations


In [11]:
# ======================================
# Cell 7: Display Recommendations
# ======================================

print("Top Recommendations for User 1:")
recommend_items(1)

Top Recommendations for User 1:


itemId
C   -5.551115e-16
D   -9.992007e-16
Name: 1, dtype: float64